**Общая информация**

**Срок сдачи:** 18 октября 2017, 06:00 <br\>
**Штраф за опоздание:** -2 балла после 06:00 18 октября, -4 балла после 06:00 25 октября, -6 баллов после 06:00 1 ноября

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush

Необходимо в slack создать таск в приватный чат:
/todo Фамилия Имя ссылка на гитхаб @alkhamush
Пример:
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush
Дополнительно нужно просто скинуть ссылку в slack в личный чат

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе. Score не должен уступать значению KNN из sklearn

###### Задание 2 (2 балла)
Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris и mnist
Для этого используем numpy

###### Задание 3 (2 балла)
Для iris найдите такой параметр n_neighbors, при котором выдаётся наилучший score. Нарисуйте график зависимости score от n_neighbors

###### Задание 4 (3 балла)
Добавить algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Значение n_neighbors нужно взять из задания 3. Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris и mnist
Для этого используем numpy. Score не должен уступать значению KNN из sklearn

###### Задание 5 (1 балла)
Описать для чего нужны следующие библиотеки/классы/функции (список будет ниже)

**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять структуру кода и текст. В противном случае -1 балл


In [44]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata
from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [45]:
%%pycodestyle

class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin,
                             SupervisedIntegerMixin, ClassifierMixin):

    def __init__(self, n_neighbors, algorithm='brute'):
        self.neighbors = n_neighbors
        self.algorithm = algorithm

    def fit(self, X, y):
        self.x_train = np.array(X).astype(float)
        self.y_train = np.array(y)
        self.y_label = np.unique(y)
        self.n_classes = np.shape(self.y_label)[0]
        self.y_dist = (self.x_train ** 2).sum(axis=1)
        if self.algorithm == 'kd_tree':
            self.tree = KDTree(self.x_train, leaf_size=60)

    def predict(self, X):
        if self.algorithm == 'brute':
            return self.predict_brute(X)
        elif self.algorithm == 'kd_tree':
            return self.predict_kdtree(X)
        
    def predict_proba(self, X):
        if self.algorithm == 'brute':
            return self.predict_proba_brute(X)
        elif self.algorithm == 'kd_tree':
            return self.predict_proba_kdtree(X)

    def predict_kdtree(self, X):
        X = X.astype(float)
        dist, ind = self.tree.query(X, k=self.neighbors)
        predict = self.y_train[ind].T
        return stats.mode(predict)[0].reshape(-1)

    def predict_brute(self, X):
        X = X.astype(float)
        x_dist = (X ** 2).sum(axis=1)
        xy_dist = X.dot(self.x_train.T)
        dist = -2*xy_dist + self.y_dist + x_dist.reshape(-1, 1)
        distance = np.sqrt(dist)
        predict = self.y_train[np.argsort(distance)[:, :self.neighbors]]
        return stats.mode(predict.T)[0].reshape(-1)
    
    def predict_proba_kdtree(self, X):
        X = X.astype(float)
        dist, k_pred = self.tree.query(X, k=self.neighbors)
        predict = np.zeros((np.shape(X)[0], np.shape(self.y_label)[0]))
        for i in range(np.shape(k_pred)[0]):
            for j in k_pred[i]:
                predict[i][int(self.y_train[j])] += 1
        return predict / self.neighbors

    def predict_proba_brute(self, X):
        X = X.astype(float)
        x_dist = (X ** 2).sum(axis=1)
        xy_dist = X.dot(self.x_train.T)
        dist = -2*xy_dist + self.y_dist + x_dist.reshape(-1, 1)
        distance = np.sqrt(dist)

        k_pred = np.argsort(distance)[:, :self.neighbors]
        predict = np.zeros((np.shape(X)[0], np.shape(self.y_label)[0]))
        for i in range(np.shape(k_pred)[0]):
            for j in k_pred[i]:
                predict[i][int(self.y_train[j])] += 1
        return predict / self.neighbors

    def score(self, X, y):
        y_pred = self.predict(X)
        return np.array([1 for i, x in enumerate(y_pred)
                        if x == y[i]]).sum(axis=0) / len(y)

3:1: E302 expected 2 blank lines, found 1
24:1: W293 blank line contains whitespace
45:1: W293 blank line contains whitespace


**IRIS**

In [46]:
iris = datasets.load_iris()

In [47]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [48]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [49]:
%time clf.fit(X_train, y_train)

CPU times: user 950 µs, sys: 552 µs, total: 1.5 ms
Wall time: 924 µs


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [50]:
%time my_clf.fit(X_train, y_train)

CPU times: user 465 µs, sys: 167 µs, total: 632 µs
Wall time: 478 µs


In [51]:
%time clf.predict(X_test)

CPU times: user 909 µs, sys: 162 µs, total: 1.07 ms
Wall time: 922 µs


array([0, 0, 0, 1, 1, 2, 1, 0, 1, 1, 2, 2, 0, 2, 2])

In [20]:
%time my_clf.predict(X_test)

AttributeError: 'MyKNeighborsClassifier' object has no attribute 'neighbors'

In [ ]:
%time clf.predict_proba(X_test)

In [ ]:
%time my_clf.predict_proba(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
my_clf.score(X_test, y_test)

Задание 3

**MNIST**

In [ ]:
mnist = fetch_mldata('MNIST original')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.01, stratify=mnist.target)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [ ]:
%time clf.fit(X_train, y_train)

In [ ]:
%time my_clf.fit(X_train, y_train)

In [ ]:
%time clf.predict(X_test)

In [ ]:
%time my_clf.predict(X_test)

In [ ]:
%time clf.predict_proba(X_test)

In [ ]:
%time my_clf.predict_proba(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
my_clf.score(X_test, y_test)

Задание 4

In [ ]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.01, stratify=mnist.target)

In [ ]:
%time clf.fit(X_train, y_train)

In [ ]:
%time my_clf.fit(X_train, y_train)

In [ ]:
%time clf.predict(X_test)

In [ ]:
%time my_clf.predict(X_test)

In [ ]:
%time clf.predict_proba(X_test)

In [ ]:
%time my_clf.predict_proba(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
my_clf.score(X_test, y_test)

Задание 5

In [ ]:
# seaborn
# matplotlib
# train_test_split
# Pipelin%load_ext e (from sklearn.pipeline import Pipeline)
# StandardScaler (from sklearn.preprocessing import StandardScaler)
# ClassifierMixin
# NeighborsBase
# KNeighborsMixin
# SupervisedIntegerMixin